# [RL - Exercise](https://fhict.instructure.com/courses/13057/assignments/219963?module_item_id=921380)

### Relevant Modules
[RL - How to apply?](https://fhict.instructure.com/courses/13057/pages/rl-how-to-apply?module_item_id=921379) <br/>
[RL - Theoretical background](https://fhict.instructure.com/courses/13057/pages/rl-theoretical-background?module_item_id=921378) <br/>


### Description
Now you finished the assignments working with Gym it's time for the real thing.

Find a game you like and create a reinforcement agent that can play this game. This can be any game, but remember that you have to be able to access or being able to calculate the three main parameters of reinforcement learning: state, action and reward. If you want to play the Gym Atari games, tak a look at this [dead link](docker) docker container.

### Deliverable
Hand in a research report that contains a link to your documented code in Git (the school's GitlabLinks to an external site.), the results and methods you used to achieve the results.
 

#################################################################

# Code



## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # or any {‘0’, ‘1’, ‘2’}

import tensorflow as tf
from tensorflow import keras

import gymnasium as gym

import pandas as pd

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

import numpy as np
import seaborn as sns

from datetime import datetime
import time
import random

######################

print(tf.__version__)
print(keras.__version__)
print(gym.__version__)
print(matplotlib.__version__)
print(pd.__version__)
print(np.__version__)
print(sns.__version__)

2.12.0
2.12.0
0.28.1
3.7.1
1.5.3
1.23.5
0.12.2


## GPU

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(f"Is built with CUDA {tf.test.is_built_with_cuda()}")

Num GPUs Available:  1
Is built with CUDA True


## Data

In [3]:
# try:
#     os.environ["DISPLAY"]
#     print("Succes")
# except:
#     os.environ["SDL_VIDEODRIVER"] = "dummy"
#     print("Dummy")

## Model

In [4]:
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)

for _ in range(1000):
    action = env.action_space.sample()  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

error: No available video device

## Evaluation

# Appendix



## Issues


- ...........


## Sources



[Title of example](https://www.google.com)

















##### Example, ToDo Delete

[Tensorflow docs; loading Fasion_MNIST](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data)

[Udemy, Intro to Machine Learning, Decision Trees](https://learn.udacity.com/courses/ud120/lessons/45da7686-9785-44db-9cbc-55a0286292a3/concepts/337a88e5-b117-494f-ac66-574cdf39a35b)

[SciKit Decision Tree blogpost](https://towardsdatascience.com/scikit-learn-decision-trees-explained-803f3812290d)

[scikit learn; RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[scikit learn; ensemble](https://scikit-learn.org/stable/modules/ensemble.html#forest)

[scikit learn; PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA)

[scikit learn; logistic regression (with softmax)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

[Canvas, Decision Trees](https://fhict.instructure.com/courses/13057/pages/decision-trees)

[ADS-A ML w5 Decision Trees](../Files/ADS-A%20ML%20w5%20Decision%20Trees.pdf)

[Numpy reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)

